In [1]:
import pandas as pd
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [3]:
df = pd.read_csv('court-V-N.csv') 
rows = list(map(list, df.values)) 
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)

In [5]:
# золотой стандарт

golden_stand = [(('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'), 1),
                (('УДОВЛЕТВОРИТЬ', 'ИСК'), 2),
                (('СУД', 'УДОВЛЕТВОРИТЬ'), 3), 
                (('НАЗНАЧИТЬ','НАКАЗАНИЕ'), 4), 
                (('ПРИЗНАТЬ','СООБЩНИК'), 5), 
                (('УДОВЛЕТВОРИТЬ','ИСК'), 6), 
                (('НАЛОЖИТЬ','АРЕСТ'), 7), 
                (('ОТКЛОНИТЬ','ИСК'), 8), 
                (('САНКЦИЯ','АРЕСТ'), 9), 
                (('ПРЕКРАТИТЬ','ДЕЛО'), 10)]

In [11]:
# likelihood ratio

top_lr = finder.nbest(bigram_measures.likelihood_ratio, 10)
list_with_scores_lr = list(finder.score_ngrams(bigram_measures.likelihood_ratio))
list_no_scores_lr = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_lr]
top_lr[:10]

[('СУД ', 'ПРИЗНАТЬ '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('СУД ', 'УДОВЛЕТВОРИТЬ '),
 ('УДОВЛЕТВОРИТЬ ', 'ИСК '),
 ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО '),
 ('ПРИНЯТЬ ', 'РЕШЕНИЕ '),
 ('САНКЦИЯ ', 'АРЕСТ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('НАЛОЖИТЬ ', 'АРЕСТ '),
 ('СУД ', 'ВЫНЕСТИ ')]

In [12]:
# PMI

top_pmi = finder.nbest(bigram_measures.pmi, 10)
list_with_scores_pmi = list(finder.score_ngrams(bigram_measures.pmi))
list_no_scores_pmi = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_pmi]
top_pmi[:10]

[(' АКЦИОНЕР ', 'РОЗНИЧНЫЙ '),
 (' АРМИЯ ', 'ВОССТАНОВИТЬСЯ '),
 (' БОЛЬШИНСТВО ', 'СООТЕЧЕСТВЕННИК '),
 (' БРИГАДИР ', 'ВИКТОР '),
 (' ВЕСТИ ', 'ПЕРЕГОВОРЫ '),
 (' ВЗЯТЬ ', 'РЕАЛИЗАЦИЯ '),
 (' ВОЗГЛАВИТЬ ', 'СПИСОК '),
 (' ВРЕМЯ ', 'КНЯЗЕВ '),
 (' ВСПОМИНАТЬСЯ ', 'ПОПЫТКА '),
 (' ДОЛЛАР ', 'ПОТРАТИТЬ ')]

In [14]:
ranks_golden = [p[1] for p in golden_stand]
ranks_lr = [list_no_scores_lr.index(i[0]) for i in golden_stand]
ranks_pmi = [list_no_scores_pmi.index(i[0]) for i in golden_stand]
print('gold', ranks_golden)
print('lr', ranks_lr)
print('pmi', ranks_pmi)

# здесь видно, что только топ коллокаций likelihood ratio как-то совпадает с золотым стандартом, pmi сильно отличается

gold [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lr [4, 3, 2, 1992, 2146, 3, 8, 80, 6, 1648]
pmi [1924, 1997, 2449, 1792, 2007, 1997, 1700, 2062, 1750, 2085]


In [16]:
spearmanr(ranks_golden,ranks_lr)

# коэффициент 0.4 подтверждает наблюдение выше, между likelihood ratio и золотым стандартом есть корреляция

SpearmanrResult(correlation=0.40729671426564323, pvalue=0.24271027029946393)

In [18]:
spearmanr(ranks_golden,ranks_pmi)

# т.к. коэффициент 0.02 > 0 можно сделать вывод, что совсем небольшая корреляция между pmi и золотым стандартом 
# всё-таки есть (по крайней  мере не отрицательная)

SpearmanrResult(correlation=0.018237166310401939, pvalue=0.96011946449672703)